In [2]:
import os

from PIL import Image
import skimage.io
import numpy as np

In [20]:
def psnr(original, decoded):
    mse = np.mean(np.power(original - decoded, 2))
    if(mse == 0):
        return 100
    max_val = 255.0
    psnr = 20 * np.log10(max_val / np.sqrt(mse))
    return psnr

In [21]:
B = 6
model_name = "default"

orig_img_np = None
my_decoded_img_np = None

for filename in ["lena", "baboon", "peppers"]:
    image_path = fr"compression-lab-imgs\{filename}.png"
    encode_output_path = fr"compression-lab-imgs\{filename}_B{B}.neural"
    compressor_state_path = fr"compression-lab-imgs\{filename}_B{B}_state.json"
    decode_output_path = fr"compression-lab-imgs\{filename}_decoder_output.bmp"

    orig_img_np = skimage.io.imread(image_path)
    n_pixels = np.prod(orig_img_np.shape)

    !python encode.py -i {image_path} -B {B} -s {compressor_state_path} -o {encode_output_path} -m {model_name}
    !python decode.py -i {encode_output_path} -B {B} -s {compressor_state_path} -o {decode_output_path}  -m {model_name}

    # прочитать выход decode
    my_img_np = skimage.io.imread(decode_output_path)
    my_psnr = psnr(orig_img_np, my_img_np)
    my_bpp = os.path.getsize(encode_output_path) / n_pixels

    orig_img = Image.fromarray(orig_img_np)    
    for jpg_quality in [30]:
        jpg_path = fr"compression-lab-imgs\{filename}_q{jpg_quality}.jpg"
        orig_img.save(jpg_path, "JPEG", quality=jpg_quality, optimize=True, progressive=True)
        
        jpg_np = skimage.io.imread(jpg_path)
        jpg_psnr = psnr(orig_img_np, jpg_np)
        jpg_bpp = os.path.getsize(jpg_path) / n_pixels